In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

In [0]:
def generate_time_series(batch_size, n_steps):
  freq1, freq2, offsets1, offsets2 = np.random.rand(4,batch_size,1)
  time = np.linspace(0,1,n_steps)
  series = 0.5 * np.sin((time - offsets1)*(freq1*10+10))
  series += 0.2 * np.sin((time - offsets2)*(freq2*20+20))
  series += 0.1 * (np.random.rand(batch_size, n_steps)-0.5)
  return series[...,np.newaxis].astype(np.float32)

In [0]:
n_steps = 50
series = generate_time_series(10000,n_steps+10)
X_train, y_train = series[:7000,:n_steps], series[:7000,-10:,0]
X_valid,y_valid = series[7000:9000,:n_steps], series[7000:9000,-10:,0]
X_test,y_test = series[9000:,:n_steps], series[9000:,-10:,0]

In [0]:
Y = np.empty((10000,n_steps,10))
for step_ahead in range(1,10+1):
  Y[:,:,step_ahead-1] = series[:,step_ahead:step_ahead+n_steps, 0]
Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]

# LSTM

In [0]:
model = keras.models.Sequential(
    [
     keras.layers.LSTM(20,return_sequences=True,input_shape=[None,1]),
     keras.layers.LSTM(20,return_sequences=True),
     keras.layers.TimeDistributed(keras.layers.Dense(10))
    ]
)

In [0]:
def last_time_step(Y_true,Y_pred):
  return keras.metrics.mean_squared_error(Y_true[:,-1],Y_pred[:,-1])

In [0]:
model.compile(
    loss = "mse",
    optimizer="adam",
    metrics = [last_time_step]
)

In [9]:
model.fit(X_train,Y_train,epochs=20,validation_data=(X_valid,Y_valid))

Epoch 1/20
219/219 [==============================] - 9s 41ms/step - loss: 0.0748 - last_time_step: 0.0603 - val_loss: 0.0507 - val_last_time_step: 0.0287
Epoch 2/20
219/219 [==============================] - 8s 39ms/step - loss: 0.0465 - last_time_step: 0.0254 - val_loss: 0.0412 - val_last_time_step: 0.0202
Epoch 3/20
219/219 [==============================] - 8s 38ms/step - loss: 0.0376 - last_time_step: 0.0174 - val_loss: 0.0345 - val_last_time_step: 0.0152
Epoch 4/20
219/219 [==============================] - 8s 38ms/step - loss: 0.0329 - last_time_step: 0.0139 - val_loss: 0.0315 - val_last_time_step: 0.0125
Epoch 5/20
219/219 [==============================] - 8s 38ms/step - loss: 0.0306 - last_time_step: 0.0125 - val_loss: 0.0295 - val_last_time_step: 0.0116
Epoch 6/20
219/219 [==============================] - 8s 38ms/step - loss: 0.0290 - last_time_step: 0.0114 - val_loss: 0.0281 - val_last_time_step: 0.0104
Epoch 7/20
219/219 [==============================] - 8s 38ms/step - l

# LSTM - Peephole Connections

In [0]:
model = keras.models.Sequential(
    [
     keras.layers.RNN(keras.experimental.PeepholeLSTMCell(20),return_sequences=True,input_shape=[None,1]),
     keras.layers.RNN(keras.experimental.PeepholeLSTMCell(20),return_sequences=True),
     keras.layers.TimeDistributed(keras.layers.Dense(10))
    ]
)

In [0]:
model.compile(
    loss = "mse",
    optimizer="adam",
    metrics = [last_time_step]
)

In [12]:
model.fit(X_train,Y_train,epochs=20,validation_data=(X_valid,Y_valid))

Epoch 1/20
219/219 [==============================] - 24s 108ms/step - loss: 0.0778 - last_time_step: 0.0631 - val_loss: 0.0530 - val_last_time_step: 0.0336
Epoch 2/20
219/219 [==============================] - 23s 106ms/step - loss: 0.0447 - last_time_step: 0.0247 - val_loss: 0.0393 - val_last_time_step: 0.0198
Epoch 3/20
219/219 [==============================] - 23s 105ms/step - loss: 0.0373 - last_time_step: 0.0181 - val_loss: 0.0352 - val_last_time_step: 0.0162
Epoch 4/20
219/219 [==============================] - 23s 105ms/step - loss: 0.0339 - last_time_step: 0.0151 - val_loss: 0.0322 - val_last_time_step: 0.0133
Epoch 5/20
219/219 [==============================] - 23s 105ms/step - loss: 0.0313 - last_time_step: 0.0127 - val_loss: 0.0302 - val_last_time_step: 0.0120
Epoch 6/20
219/219 [==============================] - 23s 105ms/step - loss: 0.0294 - last_time_step: 0.0114 - val_loss: 0.0287 - val_last_time_step: 0.0113
Epoch 7/20
219/219 [==============================] - 23s 

# GRU

In [0]:
model = keras.models.Sequential(
    [
     keras.layers.GRU(20,return_sequences=True,input_shape=[None,1]),
     keras.layers.GRU(20,return_sequences=True),
     keras.layers.TimeDistributed(keras.layers.Dense(10))
    ]
)

In [0]:
model.compile(
    loss = "mse",
    optimizer="adam",
    metrics = [last_time_step]
)

In [15]:
model.fit(X_train,Y_train,epochs=20,validation_data=(X_valid,Y_valid))

Epoch 1/20
219/219 [==============================] - 10s 45ms/step - loss: 0.0789 - last_time_step: 0.0712 - val_loss: 0.0521 - val_last_time_step: 0.0406
Epoch 2/20
219/219 [==============================] - 9s 43ms/step - loss: 0.0471 - last_time_step: 0.0368 - val_loss: 0.0441 - val_last_time_step: 0.0326
Epoch 3/20
219/219 [==============================] - 10s 45ms/step - loss: 0.0425 - last_time_step: 0.0315 - val_loss: 0.0401 - val_last_time_step: 0.0286
Epoch 4/20
219/219 [==============================] - 9s 43ms/step - loss: 0.0384 - last_time_step: 0.0276 - val_loss: 0.0364 - val_last_time_step: 0.0251
Epoch 5/20
219/219 [==============================] - 9s 43ms/step - loss: 0.0354 - last_time_step: 0.0253 - val_loss: 0.0337 - val_last_time_step: 0.0227
Epoch 6/20
219/219 [==============================] - 9s 43ms/step - loss: 0.0333 - last_time_step: 0.0230 - val_loss: 0.0319 - val_last_time_step: 0.0218
Epoch 7/20
219/219 [==============================] - 9s 43ms/step -

# GRU + 1D Conv

In [0]:
model = keras.models.Sequential(
    [
     keras.layers.Conv1D(filters=20,kernel_size=4,strides=2,padding='VALID',input_shape=[None,1]),
     keras.layers.GRU(20,return_sequences=True),
     keras.layers.GRU(20,return_sequences=True),
     keras.layers.TimeDistributed(keras.layers.Dense(10))
    ]
)

In [0]:
model.compile(
    loss = "mse",
    optimizer="adam",
    metrics = [last_time_step]
)

In [18]:
model.fit(X_train,Y_train[:,3::2],epochs=20,validation_data=(X_valid,Y_valid[:,3::2]))

Epoch 1/20
219/219 [==============================] - 6s 27ms/step - loss: 0.0718 - last_time_step: 0.0650 - val_loss: 0.0467 - val_last_time_step: 0.0386
Epoch 2/20
219/219 [==============================] - 5s 23ms/step - loss: 0.0397 - last_time_step: 0.0325 - val_loss: 0.0346 - val_last_time_step: 0.0266
Epoch 3/20
219/219 [==============================] - 5s 23ms/step - loss: 0.0329 - last_time_step: 0.0249 - val_loss: 0.0299 - val_last_time_step: 0.0207
Epoch 4/20
219/219 [==============================] - 5s 24ms/step - loss: 0.0288 - last_time_step: 0.0193 - val_loss: 0.0269 - val_last_time_step: 0.0167
Epoch 5/20
219/219 [==============================] - 5s 24ms/step - loss: 0.0260 - last_time_step: 0.0157 - val_loss: 0.0250 - val_last_time_step: 0.0147
Epoch 6/20
219/219 [==============================] - 5s 24ms/step - loss: 0.0245 - last_time_step: 0.0141 - val_loss: 0.0234 - val_last_time_step: 0.0129
Epoch 7/20
219/219 [==============================] - 5s 24ms/step - l